In [4]:
import requests
import pandas as pd

def fetch_bpce_data(rows=1000):
    url = "https://bpce.opendatasoft.com/api/records/1.0/search/"
    params = {
        "dataset": "agences-caisse-depargne0",
        "rows": rows
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    records = response.json()["records"]

    data = []
    for r in records:
        f = r["fields"]
        data.append({
            "banque": "Caisse d'Épargne",
            "nom": f.get("nom_agence"),
            "adresse": f.get("adresse1") or f.get("adresse"),
            "code_postal": f.get("code_postal"),
            "latitude": f.get("coordonnees", [None, None])[0],
            "longitude": f.get("coordonnees", [None, None])[1],
        })

    return pd.DataFrame(data)

if __name__ == "__main__":
    df = fetch_bpce_data(5000)
    df.to_csv("bpce_agences_min.csv", index=False)
    print("✅ Données enregistrées dans bpce_agences_min.csv")


✅ Données enregistrées dans bpce_agences_min.csv


In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

def fetch_all_bnp(city="Paris", max_pages=5):
    url = "https://nos-agences.mabanque.bnpparibas/resultats"
    headers = {
        "X-Requested-With": "XMLHttpRequest",
        "User-Agent": "Mozilla/5.0"
    }

    all_agences = []

    for page in range(1, max_pages + 1):
        params = {
            "q": city,
            "s": "autocomplete",
            "_xhr": 1,
            "page": page,
            "b": "48.75,2.15,49.00,2.55"

        }

        print(f"🔄 Page {page}")
        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print("❌ Erreur de requête sur page", page)
            break

        try:
            data = response.json()
        except Exception:
            print("⚠️ JSON non valide, fin de boucle.")
            break

        html = data["results"]["items"]
        soup = BeautifulSoup(html, "html.parser")

        for article in soup.select("article.b-result"):
            lat = article.get("data-lat")
            lng = article.get("data-lng")

            nom_tag = article.select_one("h2.b-result__name > a")
            adresse_tags = article.select("p.b-result__address")

            if nom_tag and len(adresse_tags) >= 2:
                nom = nom_tag.text.strip()
                adresse_ligne1 = adresse_tags[0].text.strip()
                adresse_ligne2 = adresse_tags[1].text.strip()
                adresse = f"{adresse_ligne1}, {adresse_ligne2}"
                code_postal = adresse_ligne2.split()[0]

                all_agences.append({
                    "banque": "BNP Paribas",
                    "nom": nom,
                    "adresse": adresse,
                    "code_postal": code_postal,
                    "latitude": lat,
                    "longitude": lng
                })

        # Pause légère pour éviter d’être bloqué
        time.sleep(0.8)

    return pd.DataFrame(all_agences)

if __name__ == "__main__":
    df = fetch_all_bnp("", max_pages=3)
  # Tu peux mettre jusqu'à 211 pages
    df.to_csv("bnp_paris_all.csv", index=False)
    print("✅ BNP Paris —", len(df), "agences collectées")


🔄 Page 1
🔄 Page 2
🔄 Page 3
✅ BNP Paris — 30 agences collectées


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    # Ne pas activer le mode headless pour que tout se charge correctement
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(616, 970)  # Taille trouvée via ton enregistrement
    driver.set_page_load_timeout(60)
    return driver

def click_voir_plus_until_end(driver, wait_time=2, max_clicks=100):
    wait = WebDriverWait(driver, 10)
    for i in range(max_clicks):
        try:
            btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Voir plus']")))
            print(f"🟢 Clic {i+1} sur 'Voir plus'")
            driver.execute_script("arguments[0].scrollIntoView(true);", btn)
            time.sleep(0.5)
            btn.click()
            time.sleep(wait_time)
        except:
            print("✅ Plus de bouton 'Voir plus'")
            break

def extract_agences_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    agences = []

    for article in soup.select("article.b-result"):
        lat = article.get("data-lat")
        lng = article.get("data-lng")
        nom_tag = article.select_one("h2.b-result__name")
        adresse_tags = article.select("p.b-result__address")

        if nom_tag and len(adresse_tags) >= 2:
            nom = nom_tag.text.strip()
            adresse_ligne1 = adresse_tags[0].text.strip()
            adresse_ligne2 = adresse_tags[1].text.strip()
            adresse = f"{adresse_ligne1}, {adresse_ligne2}"
            code_postal = adresse_ligne2.split()[0]

            agences.append({
                "nom": nom,
                "adresse": adresse,
                "code_postal": code_postal,
                "latitude": lat,
                "longitude": lng
            })

    return agences

def scrape_region(region_slug, output_dir="agences_bnp"):
    url = f"https://nos-agences.mabanque.bnpparibas/{region_slug}"
    driver = setup_driver()

    print(f"\n🌍 Région : {region_slug}")
    driver.get(url)
    
    
    try:
        wait = WebDriverWait(driver, 5)
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button")))
        accept_btn.click()
        print("🍪 Cookies acceptés")
        time.sleep(1)
    except:
        print("ℹ️ Pas de bannière de cookies détectée (ou déjà acceptée)")


    click_voir_plus_until_end(driver)
    html = driver.page_source
    driver.quit()

    agences = extract_agences_from_html(html)
    df = pd.DataFrame(agences)
    df.drop_duplicates(subset=["nom", "adresse"], inplace=True)

    os.makedirs(output_dir, exist_ok=True)
    path = os.path.join(output_dir, f"{region_slug}.csv")
    df.to_csv(path, index=False)
    print(f"✅ {len(df)} agences sauvegardées dans {path}")

def scrape_all_regions():
    regions = [
        "bourgogne-franche-comte",
        "ile-de-france",
        "hauts-de-france",
        "occitanie",
        "bretagne",
        "auvergne-rhone-alpes",
        "nouvelle-aquitaine",
        "pays-de-la-loire",
        "provence-alpes-cote-d-azur",
        "grand-est",
        "normandie",
        "centre-val-de-loire"
    ]
    for region in regions:
        scrape_region(region)

if __name__ == "__main__":
    scrape_all_regions()



🌍 Région : ile-de-france
ℹ️ Pas de bannière de cookies détectée (ou déjà acceptée)
🟢 Clic 1 sur 'Voir plus'
🟢 Clic 2 sur 'Voir plus'
🟢 Clic 3 sur 'Voir plus'
🟢 Clic 4 sur 'Voir plus'
🟢 Clic 5 sur 'Voir plus'
🟢 Clic 6 sur 'Voir plus'
🟢 Clic 7 sur 'Voir plus'
🟢 Clic 8 sur 'Voir plus'
🟢 Clic 9 sur 'Voir plus'
🟢 Clic 10 sur 'Voir plus'
🟢 Clic 11 sur 'Voir plus'
🟢 Clic 12 sur 'Voir plus'
🟢 Clic 13 sur 'Voir plus'
🟢 Clic 14 sur 'Voir plus'
🟢 Clic 15 sur 'Voir plus'
🟢 Clic 16 sur 'Voir plus'
🟢 Clic 17 sur 'Voir plus'
🟢 Clic 18 sur 'Voir plus'
🟢 Clic 19 sur 'Voir plus'
🟢 Clic 20 sur 'Voir plus'
🟢 Clic 21 sur 'Voir plus'
🟢 Clic 22 sur 'Voir plus'
🟢 Clic 23 sur 'Voir plus'
🟢 Clic 24 sur 'Voir plus'
🟢 Clic 25 sur 'Voir plus'
🟢 Clic 26 sur 'Voir plus'
🟢 Clic 27 sur 'Voir plus'
🟢 Clic 28 sur 'Voir plus'
🟢 Clic 29 sur 'Voir plus'
🟢 Clic 30 sur 'Voir plus'
🟢 Clic 31 sur 'Voir plus'
🟢 Clic 32 sur 'Voir plus'
🟢 Clic 33 sur 'Voir plus'
🟢 Clic 34 sur 'Voir plus'
🟢 Clic 35 sur 'Voir plus'
🟢 Clic 36 sur '

In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import os

# --- Configuration des villes par région ---
regions_villes = {
    "Île-de-France": ["Paris", "Versailles", "Nanterre", "Créteil", "Saint-Denis"],
    "Hauts-de-France": ["Lille", "Amiens", "Arras", "Calais"],
    "Occitanie": ["Toulouse", "Montpellier", "Nîmes", "Perpignan"],
    "Bretagne": ["Rennes", "Brest", "Quimper", "Vannes"],
    "Auvergne-Rhône-Alpes": ["Lyon", "Grenoble", "Clermont-Ferrand", "Saint-Étienne"],
    "Nouvelle-Aquitaine": ["Bordeaux", "Limoges", "Pau", "La Rochelle"],
    "Pays de la Loire": ["Nantes", "Angers", "Le Mans", "Saint-Nazaire"],
    "Bourgogne-Franche-Comté": ["Dijon", "Besançon", "Chalon-sur-Saône"],
    "Provence-Alpes-Côte d'Azur": ["Marseille", "Nice", "Toulon", "Avignon"],
    "Grand Est": ["Strasbourg", "Nancy", "Reims", "Metz"],
    "Normandie": ["Caen", "Rouen", "Le Havre"],
    "Centre-Val de Loire": ["Orléans", "Tours", "Bourges"]
}

# --- Scraping d'une seule page ---
def fetch_page_bnp(city="", page=1):
    url = "https://nos-agences.mabanque.bnpparibas/resultats"
    headers = {
        "X-Requested-With": "XMLHttpRequest",
        "User-Agent": "Mozilla/5.0"
    }

    params = {
        "q": city,
        "s": "autocomplete",
        "_xhr": 1,
        "page": page
    }

    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        return None

    try:
        data = response.json()
    except Exception:
        return None

    html = data["results"]["items"]
    soup = BeautifulSoup(html, "html.parser")

    agences = []
    for article in soup.select("article.b-result"):
        lat = article.get("data-lat")
        lng = article.get("data-lng")

        nom_tag = article.select_one("h2.b-result__name > a")
        adresse_tags = article.select("p.b-result__address")

        if nom_tag and len(adresse_tags) >= 2:
            nom = nom_tag.text.strip()
            adresse_ligne1 = adresse_tags[0].text.strip()
            adresse_ligne2 = adresse_tags[1].text.strip()
            adresse = f"{adresse_ligne1}, {adresse_ligne2}"
            code_postal = adresse_ligne2.split()[0]

            agences.append({
                "banque": "BNP Paribas",
                "nom": nom,
                "adresse": adresse,
                "code_postal": code_postal,
                "latitude": lat,
                "longitude": lng,
                "ville_recherchee": city
            })

    return pd.DataFrame(agences)

# --- Scraping cumulé et progressif ---
def scrape_bnp_by_region(regions_dict, max_pages=10, output="bnp_france_regions.csv"):
    if os.path.exists(output):
        os.remove(output)

    for region, villes in regions_dict.items():
        print(f"\n🏳️ Région : {region}")
        for ville in villes:
            print(f"📍 Ville : {ville}")
            for page in range(1, max_pages + 1):
                df = fetch_page_bnp(city=ville, page=page)
                if df is None or df.empty:
                    print(f"  ⛔ Fin à la page {page} pour {ville}")
                    break

                df.to_csv(output, mode='a', index=False, header=not os.path.exists(output))
                print(f"  ✅ Page {page} — {len(df)} agences")
                time.sleep(0.8)

    print("\n🧹 Nettoyage des doublons…")
    df = pd.read_csv(output)
    df.drop_duplicates(subset=["nom", "adresse"], inplace=True)
    df.to_csv(output, index=False)
    print(f"📦 Export terminé : {output} — {len(df)} agences uniques")

if __name__ == "__main__":
    scrape_bnp_by_region(regions_villes, max_pages=10)



🏳️ Région : Île-de-France
📍 Ville : Paris
  ✅ Page 1 — 10 agences
  ✅ Page 2 — 10 agences
  ✅ Page 3 — 10 agences
  ✅ Page 4 — 10 agences
  ✅ Page 5 — 10 agences
  ✅ Page 6 — 10 agences
  ✅ Page 7 — 10 agences
  ✅ Page 8 — 10 agences
  ✅ Page 9 — 10 agences
  ✅ Page 10 — 10 agences
📍 Ville : Versailles
  ✅ Page 1 — 10 agences
  ✅ Page 2 — 10 agences
  ✅ Page 3 — 10 agences
  ✅ Page 4 — 10 agences
  ✅ Page 5 — 10 agences
  ✅ Page 6 — 10 agences
  ✅ Page 7 — 10 agences
  ✅ Page 8 — 10 agences
  ✅ Page 9 — 10 agences
  ✅ Page 10 — 10 agences
📍 Ville : Nanterre
  ✅ Page 1 — 10 agences
  ✅ Page 2 — 10 agences
  ✅ Page 3 — 10 agences
  ✅ Page 4 — 10 agences


KeyboardInterrupt: 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.credit-agricole.fr"

def get_region_links():
    url = f"{BASE_URL}/professionnel/agence.html"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    links = []
    for a in soup.select("a[href*='/professionnel/agence/']"):
        href = a["href"]
        if href.endswith(".html") and "/ville/" not in href:
            links.append({
                "region": a.text.strip(),
                "region_url": BASE_URL + href
            })
    return links

def get_city_links(region):
    r = requests.get(region["region_url"])
    soup = BeautifulSoup(r.content, "html.parser")
    links = []
    for a in soup.select("a[href*='/ville/']"):
        links.append({
            "region": region["region"],
            "city": a.text.strip(),
            "city_url": BASE_URL + a["href"]
        })
    return links

# Récupération
regions = get_region_links()
all_cities = []
for region in regions:
    cities = get_city_links(region)
    all_cities.extend(cities)

# Sauvegarde
df = pd.DataFrame(all_cities)
df.to_csv("ca_cities_by_region.csv", index=False)
print("✅ Exporté : ca_cities_by_region.csv")


✅ Exporté : ca_cities_by_region.csv


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Charger les 5 premières villes depuis le fichier généré précédemment
df_cities = pd.read_csv("ca_cities_by_region.csv").head(5)

agences = []

for i, row in df_cities.iterrows():
    print(f"\n📍 {row['region']} — {row['city']} ({i+1}/5)")
    try:
        response = requests.get(row['city_url'])
        soup = BeautifulSoup(response.content, "html.parser")

        # Affiche le début du HTML pour inspection
        html_preview = soup.prettify()[:1500]
        print("🔎 Aperçu HTML reçu :\n", html_preview)

        # Recherche large : liste ou div contenant le mot "agence"
        found = False
        for bloc in soup.select("li, div"):
            if "agence" in bloc.get_text().lower():
                nom = bloc.get_text(strip=True)
                found = True
                agences.append({
                    "region": row["region"],
                    "ville": row["city"],
                    "nom": nom,
                    "adresse": "À extraire précisément ensuite",
                    "code_postal": "Inconnu"
                })

        if not found:
            print("⚠️ Aucune agence détectée sur cette page.")

        time.sleep(0.5)

    except Exception as e:
        print(f"❌ Erreur avec {row['city']} : {e}")

# Sauvegarde des résultats partiels
df_test = pd.DataFrame(agences)
df_test.to_csv("ca_agences_test_debug.csv", index=False)
print("\n✅ Fichier 'ca_agences_test_debug.csv' créé —", len(df_test), "agences trouvées")



📍 Alpes Provence — Aix-en-Provence (1/5)
🔎 Aperçu HTML reçu :
 <!DOCTYPE html>
<html lang="fr">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, user-scalable=no, initial-scale=1" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible">
   <meta content="/professionnel/agence/alpes-provence/ville/aix-en-provence-13090.html" property="og:url">
    <link href="/content/ca/national/npc/fr/ca.webmanifest" rel="manifest"/>
    <link href="/content/dam/assetsca/npc/logos/icone_ca_pwa.png" rel="apple-touch-icon"/>
    <meta content="Trouvez toutes les informations : horaires d'ouverture, adresse, coordonnées téléphonique de vos agences du Crédit Agricole Alpes Provence, banque et assurance à Aix-en-Provence. Prenez rendez-vous avec un conseiller pour découvrir nos produits bancaires et d'assurance à Aix-en-Provence. " name="description"/>
    <meta content="Trouvez toutes les informations : horaires d'ouverture, adresse, coordonnées téléphonique de vos 

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# Charger seulement 5 villes pour test
df_cities = pd.read_csv("ca_cities_by_region.csv").head(5)

agences = []

for i, row in df_cities.iterrows():
    print(f"\n📍 {row['region']} — {row['city']} ({i+1}/5)")

    try:
        r = requests.get(row['city_url'])
        soup = BeautifulSoup(r.content, "html.parser")
        raw_text = soup.get_text(separator="\n")
        lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

        # Balayage de groupes de 3 lignes
        for idx in range(len(lines) - 2):
            nom = lines[idx]
            adresse = lines[idx + 1]
            ville_line = lines[idx + 2]

            if re.match(r"^\d{5}\s", ville_line):  # Ex: "13090 Aix-en-Provence"
                code_postal = ville_line.split()[0]
                ville = " ".join(ville_line.split()[1:])
                agences.append({
                    "region": row["region"],
                    "ville": ville,
                    "nom": nom,
                    "adresse": f"{adresse}, {ville_line}",
                    "code_postal": code_postal
                })

        time.sleep(0.3)

    except Exception as e:
        print(f"❌ Erreur à {row['city']} : {e}")

# Sauvegarde
df_result = pd.DataFrame(agences).drop_duplicates()
df_result.to_csv("ca_agences_test_final.csv", index=False)
print(f"\n✅ Exporté : ca_agences_test_final.csv — {len(df_result)} agences extraites")



📍 Alpes Provence — Aix-en-Provence (1/5)

📍 Alpes Provence — Allauch (2/5)

📍 Alpes Provence — Apt (3/5)

📍 Alpes Provence — Arles (4/5)

📍 Alpes Provence — Aubagne (5/5)

✅ Exporté : ca_agences_test_final.csv — 22 agences extraites
